Web API:
어떤 사이트가 가지고 있는 기능을 외부에서도 쉽게 사용할 수  있게 공개한 것
XML 또는 JSON형식 등으로 응답
(API(Application Programming Interface):어떤 프로그래밍 기능을 외부의 프로그램에서 호출해서 사용할 수 있게 만든 것=서로 다른 프로그램이 기능을 공유할 수 있게 절차와 규약을 정의한 것)

In [5]:
import requests
import pprint

#API키를 지정(Encoding key로는 error가 나서 Decoding key로 진행함)
apikey='q+LmC5LyVljCscdUobqB7LjX4lt+PwuTwejcQGR+mgW0lJYVQPIe1v7kepGi8T1i/EaTH0gMyvi7WheDbzEXvA=='
url = 'http://openapi.forest.go.kr/openapi/service/forestDisasterService/frstFireOpenAPI'
params ={'serviceKey' : apikey, 'searchStDt' : '20120101', 'searchEdDt' : '20211231', 'pageNo' : '1', 'numOfRows' : '100' }

response = requests.get(url, params=params)

# xml 내용
content = response.text

# 깔끔한 출력 위한 코드
pp = pprint.PrettyPrinter(indent=4)
#print(pp.pprint(content))

### xml을 DataFrame으로 변환하기 ###
from os import name
import xml.etree.ElementTree as et
import pandas as pd
import bs4
from lxml import html
from urllib.parse import urlencode, quote_plus, unquote

## 각 컬럼 값 ## (포털 문서에서 꼭 확인하세요)
"""
SEQ : 게시글번호(국내 시도별 발생현황 고유값)
CREATE_DT: 	등록일시분초
DEATH_CNT: 	사망자 수
GUBUN: 	시도명(한글)
GUBUN_CN: 	시도명(중국어)
gubunEn: 시도명(영어)
INC_DEC: 전일대비 증감 수
ISOL_CLEAR_CNT: 격리 해제 수
QUR_RATE: 10만명당 발생률
STD_DAY: 기준일시
UPDATE_DT: 수정일시분초
DEF_CNT: 확진자 수
ISOL_ING_CNT: 격리중 환자수
OVER_FLOW_CNT: 해외유입 수
LOCAL_OCC_CNT: 지역발생 수

""" 

#bs4 사용하여 item 태그 분리

xml_obj = bs4.BeautifulSoup(content,'lxml-xml')
rows = xml_obj.findAll('item')
print(rows)
"""
# 컬럼 값 조회용
columns = rows[0].find_all()
print(columns)
"""

# 각 행의 컬럼, 이름, 값을 가지는 리스트 만들기
row_list = [] # 행값
name_list = [] # 열이름값
value_list = [] #데이터값

# xml 안의 데이터 수집
for i in range(0, len(rows)):
    columns = rows[i].find_all()
    #첫째 행 데이터 수집
    for j in range(0,len(columns)):
        if i ==0:
            # 컬럼 이름 값 저장
            name_list.append(columns[j].name)
        # 컬럼의 각 데이터 값 저장
        value_list.append(columns[j].text)
    # 각 행의 value값 전체 저장
    row_list.append(value_list)
    # 데이터 리스트 값 초기화
    value_list=[]

#xml값 DataFrame으로 만들기
corona_df = pd.DataFrame(row_list, columns=name_list)
print(corona_df.head(19)) 

#DataFrame CSV 파일로 저장
corona_df.to_csv('corona_kr.csv', encoding='utf-8')

[<item><diravg>SE</diravg><dirmax>S</dirmax><dmgarea>0.19</dmgarea><dmgmoney>0</dmgmoney><exintgtm>300</exintgtm><extingdt>201406191800</extingdt><humidcurr>0.0</humidcurr><humidmin>0.0</humidmin><humidrel>57.0</humidrel><ocurcause>쓰레기소각</ocurcause><ocurdo>강원</ocurdo><ocurdt>201406191500</ocurdt><ocuremd>남</ocuremd><ocurgm>북부</ocurgm><ocurjibun>산226-1</ocurjibun><ocurri>남전</ocurri><ocursgg>인제</ocursgg><ocuryoil>목</ocuryoil><ownersec> </ownersec><rainamount>0.5</rainamount><raindays>6</raindays><riskavg>0.0</riskavg><riskmax>0.0</riskmax><tempavg>23.6</tempavg><windavg>1.9</windavg><windmax>5.6</windmax></item>, <item><diravg>NNE</diravg><dirmax>SE</dirmax><dmgarea>0.08</dmgarea><dmgmoney>0</dmgmoney><exintgtm>210</exintgtm><extingdt>201406182015</extingdt><humidcurr>0.0</humidcurr><humidmin>0.0</humidmin><humidrel>46.0</humidrel><ocurcause>담뱃불실화 추정</ocurcause><ocurdo>경남</ocurdo><ocurdt>201406181805</ocurdt><ocuremd>삼랑진</ocuremd><ocurgm>경남</ocurgm><ocurjibun>산123-1</ocurjibun><ocurri>미전